<h1 align="center" , style="color:#B82727;">  Évaluation Comparative des Modèles de Classification Zero-Shot </h1>

## 1. Introduction 

Cette étude se concentre sur la comparaison et l'évaluation des modèles de classification zero-shot, largement utilisés sur la plateforme Hugging Face. L'objectif principal est d'analyser l'efficacité de ces modèles dans la tâche spécifique de classification des employés. Un élément clé de notre méthode consiste à utiliser divers prompts pour examiner comment ils influencent les performances de ces modèles en matière de précision et de fiabilité des prédictions. A travers cette recherche, nous cherchons à déterminer quels modèles et quels types de prompts offrent les meilleurs résultats pour cette application particulière.

In [1]:
from transformers import pipeline
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from easynmt import EasyNMT
import sentencepiece as sp

In [4]:
import time

## 2. Préparation des modèles

In [5]:
#models = ["facebook/bart-large-mnli", "MoritzLaurer/deberta-v3-large-zeroshot-v2.0","MoritzLaurer/deberta-v3-large-zeroshot-v2.0-c", "MoritzLaurer/deberta-v3-base-zeroshot-v2.0", "MoritzLaurer/roberta-large-zeroshot-v2.0","MoritzLaurer/bge-m3-zeroshot-v2.0" ]
classifier1 = pipeline("zero-shot-classification", model="facebook/bart-large-mnli" )
classifier2 = pipeline("zero-shot-classification", model ="MoritzLaurer/deberta-v3-large-zeroshot-v2.0")
classifier3 = pipeline("zero-shot-classification", model ="MoritzLaurer/deberta-v3-large-zeroshot-v2.0-c")
classifier4 = pipeline("zero-shot-classification", model ="MoritzLaurer/deberta-v3-base-zeroshot-v2.0")
classifier5 = pipeline("zero-shot-classification", model ="MoritzLaurer/roberta-large-zeroshot-v2.0")
classifier6 = pipeline("zero-shot-classification", model ="MoritzLaurer/bge-m3-zeroshot-v2.0")

## Translator 

**Pour les modèles English-only**

In [6]:
#translator.translate(text, target_lang='en')
translator = EasyNMT('opus-mt') 


## 3. Préparation et Nettoyage de données

In [7]:
avis = pd.read_csv("Gestion des changements.csv") 

In [8]:
avis.head()

,Sample_text,synergy,antagonism,class
0,Je vois clairement les avantages du nouveau sy...,Engagé,Résistant,Moteur
1,"Bonjour, Merci pour la mise à jour sur le proj...",Intéressé,Opposant,Opposant
2,Je tiens à dire combien je suis impressionné p...,Engagé,Résistant,Moteur
3,Je suis enthousiaste à l'idée d'intégrer la bl...,Coopérant,Conciliant,Aligné
4,Je tiens à souligner combien je suis investi d...,Engagé,Opposant,Déchiré


In [9]:
avis.dropna(inplace=True)


In [10]:
regions = {
        ("Conciliant","Minimaliste" ):"Passif",
        ("Conciliant", "Intéressé"): "Passif",
        ("Résistant", "Minimaliste"): "Passif",
        ("Résistant", "Intéressé"): "Passif",
        ("Conciliant", "Coopérant"): "Aligné",
        ("Conciliant", "Engagé"):"Engagé",
        ("Résistant", "Coopérant"):"Moteur",
        ("Résistant", "Engagé"): "Moteur",
        ("Opposant", "Coopérant"): "Moteur",
        ("Opposant", "Engagé"): "Moteur" ,
        ("Opposant", "Minimaliste"):"Opposant",
        ("Opposant", "Intéressé"):"Opposant",
        ("Irréconciliant", "Minimaliste"):"Irréductible",
        ("Irréconciliant", "Intéressé"):"Irréductible",
        ("Irréconciliant", "Coopérant"): "Déchiré",
        ("Irréconciliant", "Engagé"):"Déchiré"
        }
def assign_group(row):
    tuple = (row['antagonism'],row['synergy'])
    return regions[tuple]

In [11]:
avis["synergy"].unique()

array(['Engagé', 'Intéressé', 'Coopérant', 'Interessé', 'Minimaliste',
       'Engagé '], dtype=object)

In [12]:
avis["synergy"] = avis["synergy"].replace({"Interessé": "Intéressé", "Engagé ": "Engagé"})

In [13]:
avis['synergy'].value_counts()

Minimaliste    30
Engagé         27
Intéressé      24
Coopérant      19
Name: synergy, dtype: int64

In [14]:
avis["antagonism"].unique()

array(['Résistant', 'Opposant', 'Conciliant', 'Concillant',
       'Irréconciliant'], dtype=object)

In [15]:
avis["antagonism"] = avis["antagonism"].replace({"Concillant": "Conciliant"})

In [16]:
avis['antagonism'].value_counts()

Résistant         33
Conciliant        26
Opposant          24
Irréconciliant    17
Name: antagonism, dtype: int64

In [17]:
avis['class'].unique()

array(['Moteur', 'Opposant', 'Aligné', 'Déchiré', 'Passif',
       'Irréductible', 'Engagé'], dtype=object)

In [18]:
avis ["class"] = avis.apply(assign_group, axis=1)

## 4. Formulation des prompts 

In [19]:
#Francais 
#formulation 1 : formulation minimaliste
text_synergy_labels1 =["1 Minimaliste","2 Intéressé", "3 Coopérant" , "4 Engagé"] 
text_antagonism_labels1 = [ "1 Conciliant","2 Résistant" , "3 Opposant", "4 Irréconciliant" ]
#formulation 2 : formulation explicite 
text_synergy_labels2 =["1 Minimaliste  Se limite strictement à exécuter les tâches assignées avec un effort minimal N'initie aucune action au-delà de ce qui est explicitement demandé",
                        "2 Intéressé Manifeste un intérêt verbal  pour le projet, et participe sans hésitation. Engagé au niveau basique, mais sans initiative significative.",
                        "3 Coopérant S'implique de manière proactive prenant des initiatives concrètes sans attendre de directives" ,
                        "4 Engagé Démontre un engagement maximal en prenant des initiatives majeures qui influencent le cours des projets. Travaille avec passion pour inspirer et entraîner les autres"] 
text_antagonism_labels2 = [ "1 Conciliant : Adopte une approche souple et s'adapte facilement aux décisions de groupe, même si personnellement en désaccord."
                            ,"2 Résistant: Exprime ouvertement son désaccord ou ses critiques concernant les propositions ou les projets, mais reste ouvert à la discussion et peut être persuadé de soutenir la décision finale avec des ajustements." ,
                            "3 Opposant: Manifeste activement son opposition à travers ses actions ou son comportement, créant des obstacles ou des retards. Toutefois, il peut être amené à se rallier si placé sous une pression ", 
                            "4 Irréconciliant :S'oppose fermement et utilise tous les moyens à sa disposition pour contrecarrer les initiatives ou les changements proposés. Refuse de se rallier quelles que soient les circonstances ou les pressions exercées" ]
# formulation 3 : formulation optimale 
text_synergy_labels3 = ["1 fait les directives assignés sans des initiatives actives",
                        "2 fait les directives assignés et manifeste un interet verbal pour le projet mais sans initiatives actives ",
                        "3 fait des initiatives actives " ,
                        "4 fait des initiatives actives avec ton élevé de responsabilité "   ]
text_antagonism_labels3 =["1  Soutient les décisions et approches du projet",
                            "2 contre le projet ou certaines décisions ou approches  et ouvert à la discussion",
                            "3 contre le projet ou certaines décisions ou approches et moins flexible à la discussion",
                            "4 contre le projet ou certaines décisions ou approches  et inflexible  à la discussion" ]
#English
# formulation 1
text_synergy_labels1e =["1 Minimalist","2 Interested", "3 Cooperative" , "4 Engaged"]
text_antagonism_labels1e = [ "1 Conciliatory","2 Resistant" , "3 Opposing", "4 Irreconcilable" ]
#formulation 2 
text_synergy_labels2e =["1 Minimalist: Strictly limits themselves to executing assigned tasks with minimal effort. Does not initiate any actions beyond what is explicitly requested.",
                        "2 Interested: Verbally expresses interest in the project and participates without hesitation. Engaged at a basic level, but without significant initiative.",
                        "3 Cooperative: Actively engages by taking concrete initiatives without waiting for directives.",
                        "4 Engaged: Demonstrates maximum commitment by taking major initiatives that influence the course of projects. Works passionately to inspire and lead others."]

text_antagonism_labels2e = [ "1 Conciliatory: Adopts a flexible approach and easily adapts to group decisions, even if personally in disagreement.",
                             "2 Resistant: Openly expresses disagreement or criticism regarding proposals or projects but remains open to discussion and can be persuaded to support the final decision with adjustments.",
                             "3 Opposing: Actively opposes through actions or behavior, creating obstacles or delays. However, may be persuaded to join if placed under sufficient pressure.",
                             "4 Irreconcilable: Firmly opposes and uses all means at their disposal to counter proposed initiatives or changes. Refuses to align under any circumstances or pressures." ]
#formulation 3
text_synergy_labels3e = ["1 performs assigned directives without active initiatives",
                         "2 performs assigned directives and expresses verbal interest in the project but without active initiatives",
                         "3 takes active initiatives",
                         "4 takes active initiatives with a high level of responsibility"]
text_antagonism_labels3e = ["1 Supports the project's decisions and approaches.",
                            "2 against the project or certain decisions or approaches and open to discussion",
                            "3 against the project or certain decisions or approaches and less flexible to discussion",
                            "4 against the project or certain decisions or approaches and inflexible to discussion"]



In [20]:
avis["Sample_text"]

0     Je vois clairement les avantages du nouveau sy...
1     Bonjour, Merci pour la mise à jour sur le proj...
2     Je tiens à dire combien je suis impressionné p...
3     Je suis enthousiaste à l'idée d'intégrer la bl...
4     Je tiens à souligner combien je suis investi d...
                            ...                        
95    L'idée du projet possède certes un certain pot...
96    je suis extrêmement enthousiaste à propos de c...
97    Je fais ce qui est demandé. Aucune initiative....
98    "L'idée est intéressante sur le papier, et j'a...
99    L'idée du projet est bien pensée, elle a du po...
Name: Sample_text, Length: 100, dtype: object

In [45]:
y_test = avis ["class"]

# Formulation minimaliste 

## Modèle 1 : facebook/bart-large-mnli"

### Tache 1 : classification selon son niveau de synergie

In [38]:
y_test_synergy= avis["synergy"]

In [24]:
y_pred_synergy11 = []

In [25]:
start = time.time()
for text in avis["Sample_text"]:
    text = translator.translate(text, target_lang='en')
    result = classifier1(text, text_synergy_labels1e, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_synergy11.append("Minimaliste")
    elif result == "2":
        y_pred_synergy11.append("Intéressé")
    elif result == "3" :
        y_pred_synergy11.append("Coopérant")
    else :
        y_pred_synergy11.append("Engagé")
end = time.time()
execution_time111 = end - start
execution_time111 = execution_time111/60

print(f'Temps d\'exécution : {execution_time111:.2f}minutes')


C:\Users\HP\anaconda3\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Temps d'exécution : 11.19minutes


In [26]:
y_pred_synergy11 = np.array(y_pred_synergy11)

In [27]:
y_pred_synergy11

array(['Intéressé', 'Intéressé', 'Intéressé', 'Intéressé', 'Intéressé',
       'Intéressé', 'Intéressé', 'Intéressé', 'Intéressé', 'Intéressé',
       'Minimaliste', 'Intéressé', 'Intéressé', 'Intéressé', 'Intéressé',
       'Engagé', 'Intéressé', 'Intéressé', 'Minimaliste', 'Engagé',
       'Intéressé', 'Intéressé', 'Minimaliste', 'Engagé', 'Engagé',
       'Intéressé', 'Intéressé', 'Intéressé', 'Intéressé', 'Intéressé',
       'Intéressé', 'Intéressé', 'Intéressé', 'Intéressé', 'Intéressé',
       'Intéressé', 'Intéressé', 'Intéressé', 'Intéressé', 'Intéressé',
       'Minimaliste', 'Intéressé', 'Intéressé', 'Intéressé', 'Intéressé',
       'Intéressé', 'Intéressé', 'Intéressé', 'Intéressé', 'Intéressé',
       'Intéressé', 'Intéressé', 'Intéressé', 'Intéressé', 'Intéressé',
       'Intéressé', 'Intéressé', 'Intéressé', 'Intéressé', 'Intéressé',
       'Intéressé', 'Intéressé', 'Intéressé', 'Engagé', 'Intéressé',
       'Intéressé', 'Intéressé', 'Intéressé', 'Intéressé', 'Intéressé',

In [28]:
cm = confusion_matrix(y_test_synergy, y_pred_synergy11, labels = ["Minimaliste", "Intéressé", "Coopérant","Engagé"])
print(cm)
print("Précision:", precision_score(y_test_synergy, y_pred_synergy11, average='macro'))
print("Rappel:", recall_score(y_test_synergy, y_pred_synergy11, average='macro'))
print("Score F1:", f1_score(y_test_synergy, y_pred_synergy11, average='macro'))
print("Accuracy:", accuracy_score(y_test_synergy, y_pred_synergy11))

[[ 6 21  0  3]
 [ 0 24  0  0]
 [ 0 17  0  2]
 [ 0 25  0  2]]
Précision: 0.39039408866995073
Rappel: 0.3185185185185185
Score F1: 0.2208532061473238
Accuracy: 0.32


C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Tache 2 : classification selon son niveau d'antagonisme

In [39]:
y_test_antagonism = avis["antagonism"]

In [30]:
y_pred_antagonism11 = []

In [31]:
start = time.time()
for text in avis["Sample_text"]:
    text = translator.translate(text, target_lang='en')
    result = classifier1(text, text_antagonism_labels1e, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_antagonism11.append("Conciliant")
    elif result == "2":
        y_pred_antagonism11.append("Résistant")
    elif result == "3" :
        y_pred_antagonism11.append("Opposant")
    else :
        y_pred_antagonism11.append("Irréconciliant")
end = time.time()
execution_time112 = end - start
execution_time112 = execution_time112/60

print(f'Temps d\'exécution : {execution_time112:.2f}minutes')

Temps d'exécution : 11.84minutes


In [32]:
y_pred_antagonism11 = np.array(y_pred_antagonism11)

In [33]:
cm2 = confusion_matrix(y_test_antagonism, y_pred_antagonism11, labels = ["Conciliant", "Résistant", "Opposant","Irréconciliant"])
print(cm2)
print("Précision:", precision_score(y_test_antagonism, y_pred_antagonism11, average='macro'))
print("Rappel:", recall_score(y_test_antagonism, y_pred_antagonism11, average='macro'))
print("Score F1:", f1_score(y_test_antagonism, y_pred_antagonism11, average='macro'))
print("Accuracy:", accuracy_score(y_test_antagonism, y_pred_antagonism11))

[[23  0  3  0]
 [ 8  0 25  0]
 [ 5  0 19  0]
 [ 0  4 13  0]]
Précision: 0.23888888888888887
Rappel: 0.4190705128205128
Score F1: 0.29857910906298
Accuracy: 0.42


C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [34]:
predictions_bart_large_mnli_1 = pd.DataFrame({
    'synergy': y_pred_synergy11,
    'antagonism': y_pred_antagonism11
})

### Tache 3 : classification finale

In [35]:
y_test = avis ["class"]

In [36]:
predictions_bart_large_mnli_1['class'] = predictions_bart_large_mnli_1.apply(assign_group, axis=1)

In [37]:
y_pred11 = predictions_bart_large_mnli_1 ["class"]

In [38]:
cm3 = confusion_matrix(y_test, y_pred11, labels = ["Passif", "Aligné", "Engagé","Moteur","Déchiré", "Opposant", "Irréductible"])
print(cm3)
print("Précision:", precision_score(y_test, y_pred11, average='macro'))
print("Rappel:", recall_score(y_test, y_pred11, average='macro'))
print("Score F1:", f1_score(y_test, y_pred11, average='macro'))
print("Accuracy:", accuracy_score(y_test, y_pred11))

[[15  0  1  1  0 11  0]
 [ 7  0  0  0  0  0  0]
 [ 3  0  0  0  0  2  0]
 [ 6  0  0  4  0 18  0]
 [ 2  0  0  0  0  4  0]
 [ 4  0  0  0  0 11  0]
 [ 1  0  0  1  0  9  0]]
Précision: 0.18020050125313286
Rappel: 0.20170068027210883
Score F1: 0.1434464694968897
Accuracy: 0.3


C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Modèle 2 : "MoritzLaurer/deberta-v3-large-zeroshot-v2.0"

### Tache 1 : classification selon son niveau de synergie

In [39]:
y_pred_synergy12 = []

In [41]:
start = time.time()
for text in avis["Sample_text"]:
    result = classifier2(text, text_synergy_labels1, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_synergy12.append("Minimaliste")
    elif result == "2":
        y_pred_synergy12.append("Intéressé")
    elif result == "3" :
        y_pred_synergy12.append("Coopérant")
    else :
        y_pred_synergy12.append("Engagé")
end = time.time()
execution_time121 = end - start
execution_time121 = execution_time121/60

print(f'Temps d\'exécution : {execution_time111:.2f}minutes')


Temps d'exécution : 11.19minutes


In [42]:
y_pred_synergy12 = np.array(y_pred_synergy12)

In [43]:
cm = confusion_matrix(y_test_synergy, y_pred_synergy12, labels = ["Minimaliste", "Intéressé", "Coopérant","Engagé"])
print(cm)
print("Précision:", precision_score(y_test_synergy, y_pred_synergy12, average='macro'))
print("Rappel:", recall_score(y_test_synergy, y_pred_synergy12, average='macro'))
print("Score F1:", f1_score(y_test_synergy, y_pred_synergy12, average='macro'))
print("Accuracy:", accuracy_score(y_test_synergy, y_pred_synergy12))

[[16  1  0 13]
 [ 7  8  0  9]
 [ 2  0  0 17]
 [ 0  1  0 26]]
Précision: 0.4600000000000001
Rappel: 0.4574074074074074
Score F1: 0.4044059521041618
Accuracy: 0.5


C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Tache 2 : classification selon son niveau d'antagonisme

In [44]:
y_pred_antagonism12 = []

In [45]:
start = time.time()
for text in avis["Sample_text"]:
    result = classifier2(text, text_antagonism_labels1, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_antagonism12.append("Conciliant")
    elif result == "2":
        y_pred_antagonism12.append("Résistant")
    elif result == "3" :
        y_pred_antagonism12.append("Opposant")
    else :
        y_pred_antagonism12.append("Irréconciliant")
end = time.time()
execution_time122 = end - start
execution_time122 = execution_time122/60

print(f'Temps d\'exécution : {execution_time122:.2f}minutes')

Temps d'exécution : 9.67minutes


In [46]:
y_pred_antagonism12 = np.array(y_pred_antagonism12)

In [47]:
cm2 = confusion_matrix(y_test_antagonism, y_pred_antagonism12, labels = ["Conciliant", "Résistant", "Opposant","Irréconciliant"])
print(cm2)
print("Précision:", precision_score(y_test_antagonism, y_pred_antagonism12, average='macro'))
print("Rappel:", recall_score(y_test_antagonism, y_pred_antagonism12, average='macro'))
print("Score F1:", f1_score(y_test_antagonism, y_pred_antagonism12, average='macro'))
print("Accuracy:", accuracy_score(y_test_antagonism, y_pred_antagonism12))

[[22  0  4  0]
 [13  2 17  1]
 [ 0  5 17  2]
 [ 1  4  9  3]]
Précision: 0.41365785514721687
Rappel: 0.44789095708213356
Score F1: 0.38508232872948517
Accuracy: 0.44


In [48]:
predictions_deberta_v3_large_zeroshot_v2_0_1 = pd.DataFrame({
    'synergy': y_pred_synergy12,
    'antagonism': y_pred_antagonism12
})

### Tache 3 : classification finale

In [49]:
predictions_deberta_v3_large_zeroshot_v2_0_1['class'] =predictions_deberta_v3_large_zeroshot_v2_0_1.apply(assign_group, axis=1)

In [50]:
y_pred12 = predictions_deberta_v3_large_zeroshot_v2_0_1["class"]

In [51]:
cm3 = confusion_matrix(y_test, y_pred12, labels = ["Passif", "Aligné", "Engagé","Moteur","Déchiré", "Opposant", "Irréductible"])
print(cm3)
print("Précision:", precision_score(y_test, y_pred12, average='macro'))
print("Rappel:", recall_score(y_test, y_pred12, average='macro'))
print("Score F1:", f1_score(y_test, y_pred12, average='macro'))
print("Accuracy:", accuracy_score(y_test, y_pred12))

[[ 4  0 12  3  0  9  0]
 [ 0  0  7  0  0  0  0]
 [ 0  0  5  0  0  0  0]
 [ 0  0  7 16  2  3  0]
 [ 0  0  1  4  1  0  0]
 [ 3  0  0  2  1  9  0]
 [ 2  0  0  3  1  4  1]]
Précision: 0.39030328798185937
Rappel: 0.367408781694496
Score F1: 0.26519998662855804
Accuracy: 0.36


C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Modèle 3 : MoritzLaurer/deberta-v3-large-zeroshot-v2.0-c

## Tache 1 : classification selon son niveau de synergie

In [52]:
y_pred_synergy13 = []

In [53]:
start = time.time()
for text in avis["Sample_text"]:
    result = classifier3(text, text_synergy_labels1, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_synergy13.append("Minimaliste")
    elif result == "2":
        y_pred_synergy13.append("Intéressé")
    elif result == "3" :
        y_pred_synergy13.append("Coopérant")
    else :
        y_pred_synergy13.append("Engagé")
end = time.time()
execution_time131 = end - start
execution_time131 = execution_time131/60

print(f'Temps d\'exécution : {execution_time131:.2f}minutes')

Temps d'exécution : 9.90minutes


In [54]:
y_pred_synergy13 = np.array(y_pred_synergy13)

In [55]:
cm = confusion_matrix(y_test_synergy, y_pred_synergy13, labels = ["Minimaliste", "Intéressé", "Coopérant","Engagé"])
print(cm)
print("Précision:", precision_score(y_test_synergy, y_pred_synergy13, average='macro'))
print("Rappel:", recall_score(y_test_synergy, y_pred_synergy13, average='macro'))
print("Score F1:", f1_score(y_test_synergy, y_pred_synergy13, average='macro'))
print("Accuracy:", accuracy_score(y_test_synergy, y_pred_synergy13))

[[12  1 10  7]
 [ 4 14  4  2]
 [ 2  8  1  8]
 [ 0  8  1 18]]
Précision: 0.4237663210445468
Rappel: 0.4256578947368421
Score F1: 0.4117197318810222
Accuracy: 0.45


### Tache 2 : classification selon son niveau d'antagonisme

In [56]:
y_pred_antagonism13 = []

In [57]:
start = time.time()
for text in avis["Sample_text"]:
    result = classifier3(text, text_antagonism_labels1, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_antagonism13.append("Conciliant")
    elif result == "2":
        y_pred_antagonism13.append("Résistant")
    elif result == "3" :
        y_pred_antagonism13.append("Opposant")
    else :
        y_pred_antagonism13.append("Irréconciliant")
end = time.time()
execution_time132 = end - start
execution_time132 = execution_time132/60

print(f'Temps d\'exécution : {execution_time132:.2f}minutes')

Temps d'exécution : 10.14minutes


In [58]:
y_pred_antagonism13 = np.array(y_pred_antagonism13)

In [59]:
cm2 = confusion_matrix(y_test_antagonism, y_pred_antagonism13, labels = ["Conciliant", "Résistant", "Opposant","Irréconciliant"])
print(cm2)
print("Précision:", precision_score(y_test_antagonism, y_pred_antagonism13, average='macro'))
print("Rappel:", recall_score(y_test_antagonism, y_pred_antagonism13, average='macro'))
print("Score F1:", f1_score(y_test_antagonism, y_pred_antagonism13, average='macro'))
print("Accuracy:", accuracy_score(y_test_antagonism, y_pred_antagonism13))

[[22  1  3  0]
 [ 6  4 20  3]
 [ 0  5 15  4]
 [ 0  3  8  6]]
Précision: 0.47025800286669855
Rappel: 0.4863267859591389
Score F1: 0.4543248217161261
Accuracy: 0.47


In [60]:
predictions_deberta_v3_large_zeroshot_v2_0_c_1 = pd.DataFrame({
    'synergy': y_pred_synergy13,
    'antagonism': y_pred_antagonism13
})

### Tache 3 : classification finale 

In [61]:
predictions_deberta_v3_large_zeroshot_v2_0_c_1['class'] =predictions_deberta_v3_large_zeroshot_v2_0_c_1.apply(assign_group, axis=1)

In [62]:
y_pred13 = predictions_deberta_v3_large_zeroshot_v2_0_c_1["class"]

In [63]:
cm3 = confusion_matrix(y_test, y_pred13, labels = ["Passif", "Aligné", "Engagé","Moteur","Déchiré", "Opposant", "Irréductible"])
print(cm3)
print("Précision:", precision_score(y_test, y_pred13, average='macro'))
print("Rappel:", recall_score(y_test, y_pred13, average='macro'))
print("Score F1:", f1_score(y_test, y_pred13, average='macro'))
print("Accuracy:", accuracy_score(y_test, y_pred13))

[[ 8  5  1  7  0  7  0]
 [ 5  0  2  0  0  0  0]
 [ 4  0  1  0  0  0  0]
 [ 3  0  1 13  6  5  0]
 [ 0  0  0  3  2  1  0]
 [ 2  0  0  4  1  8  0]
 [ 2  0  0  1  4  4  0]]
Précision: 0.21020931449502878
Rappel: 0.2595238095238095
Score F1: 0.22607204825249938
Accuracy: 0.32


C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Modèle 4 : MoritzLaurer/deberta-v3-base-zeroshot-v2.0

### Tache 1 : classification selon son niveau de synergie

In [64]:
y_pred_synergy14 = []

In [65]:
start = time.time()
for text in avis["Sample_text"]:
    result = classifier4(text, text_synergy_labels1, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_synergy14.append("Minimaliste")
    elif result == "2":
        y_pred_synergy14.append("Intéressé")
    elif result == "3" :
        y_pred_synergy14.append("Coopérant")
    else :
        y_pred_synergy14.append("Engagé")
end = time.time()
execution_time141 = end - start
execution_time141 = execution_time141/60

print(f'Temps d\'exécution : {execution_time141:.2f}minutes')

Temps d'exécution : 2.85minutes


In [66]:
y_pred_synergy14 = np.array(y_pred_synergy14 )

In [67]:
cm = confusion_matrix(y_test_synergy, y_pred_synergy14, labels = ["Minimaliste", "Intéressé", "Coopérant","Engagé"])
print(cm)
print("Précision:", precision_score(y_test_synergy, y_pred_synergy14, average='macro'))
print("Rappel:", recall_score(y_test_synergy, y_pred_synergy14, average='macro'))
print("Score F1:", f1_score(y_test_synergy, y_pred_synergy14, average='macro'))
print("Accuracy:", accuracy_score(y_test_synergy, y_pred_synergy14))

[[ 4 20  5  1]
 [ 3 17  3  1]
 [ 0 11  6  2]
 [ 0 16  8  3]]
Précision: 0.3845880681818182
Rappel: 0.3171418128654971
Score F1: 0.26793334191110374
Accuracy: 0.3


### Tache 2 : classification selon son niveau d'antagonisme

In [68]:
y_pred_antagonism14 = []

In [69]:
start = time.time()
for text in avis["Sample_text"]:
    result = classifier4(text, text_antagonism_labels1, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_antagonism14.append("Conciliant")
    elif result == "2":
        y_pred_antagonism14.append("Résistant")
    elif result == "3" :
        y_pred_antagonism14.append("Opposant")
    else :
        y_pred_antagonism14.append("Irréconciliant")
end = time.time()
execution_time142 = end - start
execution_time142 = execution_time142/60

print(f'Temps d\'exécution : {execution_time142:.2f}minutes')

Temps d'exécution : 3.04minutes


In [70]:
y_pred_antagonism14 = np.array(y_pred_antagonism14)

In [71]:
cm2 = confusion_matrix(y_test_antagonism, y_pred_antagonism14, labels = ["Conciliant", "Résistant", "Opposant","Irréconciliant"])
print(cm2)
print("Précision:", precision_score(y_test_antagonism, y_pred_antagonism14, average='macro'))
print("Rappel:", recall_score(y_test_antagonism, y_pred_antagonism14, average='macro'))
print("Score F1:", f1_score(y_test_antagonism, y_pred_antagonism14, average='macro'))
print("Accuracy:", accuracy_score(y_test_antagonism, y_pred_antagonism14))

[[26  0  0  0]
 [22  3  8  0]
 [11  0  9  4]
 [ 2  1  7  7]]
Précision: 0.5468982861400894
Rappel: 0.4694184491978609
Score F1: 0.40871582789686234
Accuracy: 0.45


In [72]:
predictions_deberta_v3_base_zeroshot_v2_0_1 = pd.DataFrame({
    'synergy': y_pred_synergy14,
    'antagonism': y_pred_antagonism14
})

### Tache 3 : classification finale

In [73]:
predictions_deberta_v3_base_zeroshot_v2_0_1['class'] =predictions_deberta_v3_base_zeroshot_v2_0_1.apply(assign_group, axis=1)

In [74]:
y_pred14 = predictions_deberta_v3_base_zeroshot_v2_0_1["class"]

In [75]:
cm3 = confusion_matrix(y_test, y_pred14, labels = ["Passif", "Aligné", "Engagé","Moteur","Déchiré", "Opposant", "Irréductible"])
print(cm3)
print("Précision:", precision_score(y_test, y_pred14, average='macro'))
print("Rappel:", recall_score(y_test, y_pred14, average='macro'))
print("Score F1:", f1_score(y_test, y_pred14, average='macro'))
print("Accuracy:", accuracy_score(y_test, y_pred14))

[[20  5  1  0  0  2  0]
 [ 2  4  1  0  0  0  0]
 [ 2  2  1  0  0  0  0]
 [ 9  6  2  0  2  8  1]
 [ 2  0  0  1  0  1  2]
 [ 6  1  0  1  1  6  0]
 [ 1  0  0  0  1  5  4]]
Précision: 0.2489383632240775
Rappel: 0.3213358070500928
Score F1: 0.2657424771710486
Accuracy: 0.35


## Modèle 5 : MoritzLaurer/roberta-large-zeroshot-v2.0

### Tache 1 : classification selon son niveau de synergie

In [76]:
y_pred_synergy15 = []

In [77]:
start = time.time()
for text in avis["Sample_text"]:
    text = translator.translate(text, target_lang='en')
    result = classifier5(text, text_synergy_labels1e, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_synergy15.append("Minimaliste")
    elif result == "2":
        y_pred_synergy15.append("Intéressé")
    elif result == "3" :
        y_pred_synergy15.append("Coopérant")
    else :
        y_pred_synergy15.append("Engagé")
end = time.time()
execution_time151 = end - start
execution_time151 = execution_time151/60

print(f'Temps d\'exécution : {execution_time151:.2f}minutes')


Temps d'exécution : 10.79minutes


In [78]:
y_pred_synergy15 = np.array(y_pred_synergy15 )

In [79]:
cm = confusion_matrix(y_test_synergy, y_pred_synergy15, labels = ["Minimaliste", "Intéressé", "Coopérant","Engagé"])
print(cm)
print("Précision:", precision_score(y_test_synergy, y_pred_synergy15, average='macro'))
print("Rappel:", recall_score(y_test_synergy, y_pred_synergy15, average='macro'))
print("Score F1:", f1_score(y_test_synergy, y_pred_synergy15, average='macro'))
print("Accuracy:", accuracy_score(y_test_synergy, y_pred_synergy15))

[[14  2  4 10]
 [ 4 14  0  6]
 [ 3  4  1 11]
 [ 6  2  0 19]]
Précision: 0.44198140828575616
Rappel: 0.45158382066276803
Score F1: 0.4259512502220411
Accuracy: 0.48


### Tache 2 : classification selon son niveau d'antagonisme

In [80]:
y_pred_antagonism15 = []

In [81]:
start = time.time()
for text in avis["Sample_text"]:
    text = translator.translate(text, target_lang='en')
    result = classifier5(text, text_antagonism_labels1e, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_antagonism15.append("Conciliant")
    elif result == "2":
        y_pred_antagonism15.append("Résistant")
    elif result == "3" :
        y_pred_antagonism15.append("Opposant")
    else :
        y_pred_antagonism15.append("Irréconciliant")
end = time.time()
execution_time152 = end - start
execution_time152 = execution_time152/60

print(f'Temps d\'exécution : {execution_time152:.2f}minutes')

Temps d'exécution : 10.06minutes


In [82]:
y_pred_antagonism15 = np.array(y_pred_antagonism15)

In [83]:
cm2 = confusion_matrix(y_test_antagonism, y_pred_antagonism15, labels = ["Conciliant", "Résistant", "Opposant","Irréconciliant"])
print(cm2)
print("Précision:", precision_score(y_test_antagonism, y_pred_antagonism15, average='macro'))
print("Rappel:", recall_score(y_test_antagonism, y_pred_antagonism15, average='macro'))
print("Score F1:", f1_score(y_test_antagonism, y_pred_antagonism15, average='macro'))
print("Accuracy:", accuracy_score(y_test_antagonism, y_pred_antagonism15))

[[21  5  0  0]
 [ 4 14 15  0]
 [ 0  9 14  1]
 [ 1  7  7  2]]
Précision: 0.5658119658119658
Rappel: 0.4832287810228987
Score F1: 0.4715309200603318
Accuracy: 0.51


In [84]:
predictions_roberta_large_zeroshot_v2_0_1 = pd.DataFrame({
    'synergy': y_pred_synergy15,
    'antagonism': y_pred_antagonism15
})

### Tache 3 : classification finale

In [85]:
predictions_roberta_large_zeroshot_v2_0_1['class'] =predictions_roberta_large_zeroshot_v2_0_1.apply(assign_group, axis=1)

In [86]:
y_pred15 = predictions_roberta_large_zeroshot_v2_0_1["class"]

In [87]:
cm3 = confusion_matrix(y_test, y_pred15, labels = ["Passif", "Aligné", "Engagé","Moteur","Déchiré", "Opposant", "Irréductible"])
print(cm3)
print("Précision:", precision_score(y_test, y_pred15, average='macro'))
print("Rappel:", recall_score(y_test, y_pred15, average='macro'))
print("Score F1:", f1_score(y_test, y_pred15, average='macro'))
print("Accuracy:", accuracy_score(y_test, y_pred15))

[[11  1  7  5  0  4  0]
 [ 2  1  4  0  0  0  0]
 [ 1  0  4  0  0  0  0]
 [ 1  0  2 16  1  8  0]
 [ 0  0  1  1  1  3  0]
 [ 6  0  0  5  0  4  0]
 [ 4  0  0  2  0  4  1]]
Précision: 0.48397991480450253
Rappel: 0.3473407544836116
Score F1: 0.3105341628613508
Accuracy: 0.38


## Modèle 6 : MoritzLaurer/bge-m3-zeroshot-v2.0

### Tache 1 : classification selon son niveau de synergie

In [88]:
y_pred_synergy16 = []

In [89]:
start = time.time()
for text in avis["Sample_text"]:
    result = classifier6(text, text_synergy_labels1, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_synergy16.append("Minimaliste")
    elif result == "2":
        y_pred_synergy16.append("Intéressé")
    elif result == "3" :
        y_pred_synergy16.append("Coopérant")
    else :
        y_pred_synergy16.append("Engagé")
end = time.time()
execution_time161 = end - start
execution_time161 = execution_time161/60

print(f'Temps d\'exécution : {execution_time161:.2f}minutes')

Temps d'exécution : 4.83minutes


In [90]:
y_pred_synergy16 = np.array(y_pred_synergy16 )

In [91]:
cm = confusion_matrix(y_test_synergy, y_pred_synergy16, labels = ["Minimaliste", "Intéressé", "Coopérant","Engagé"])
print(cm)
print("Précision:", precision_score(y_test_synergy, y_pred_synergy16, average='macro'))
print("Rappel:", recall_score(y_test_synergy, y_pred_synergy16, average='macro'))
print("Score F1:", f1_score(y_test_synergy, y_pred_synergy16, average='macro'))
print("Accuracy:", accuracy_score(y_test_synergy, y_pred_synergy16))

[[ 2 11  1 16]
 [ 1 21  1  1]
 [ 0  7  0 12]
 [ 0  9  3 15]]
Précision: 0.36126893939393934
Rappel: 0.37430555555555556
Score F1: 0.281770166453265
Accuracy: 0.38


### Tache 2 : classification selon son niveau d'antagonisme

In [92]:
y_pred_antagonism16 = []

In [93]:
start = time.time()
for text in avis["Sample_text"]:
    result = classifier6(text, text_antagonism_labels1, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_antagonism16.append("Conciliant")
    elif result == "2":
        y_pred_antagonism16.append("Résistant")
    elif result == "3" :
        y_pred_antagonism16.append("Opposant")
    else :
        y_pred_antagonism16.append("Irréconciliant")
end = time.time()
execution_time162 = end - start
execution_time162 = execution_time162/60

print(f'Temps d\'exécution : {execution_time162:.2f}minutes')

Temps d'exécution : 4.51minutes


In [94]:
y_pred_antagonism16 = np.array(y_pred_antagonism16)

In [95]:
cm2 = confusion_matrix(y_test_antagonism, y_pred_antagonism16, labels = ["Conciliant", "Résistant", "Opposant","Irréconciliant"])
print(cm2)
print("Précision:", precision_score(y_test_antagonism, y_pred_antagonism16, average='macro'))
print("Rappel:", recall_score(y_test_antagonism, y_pred_antagonism16, average='macro'))
print("Score F1:", f1_score(y_test_antagonism, y_pred_antagonism16, average='macro'))
print("Accuracy:", accuracy_score(y_test_antagonism, y_pred_antagonism16))

[[12  4 10  0]
 [ 2  6 14 11]
 [ 1  8  7  8]
 [ 0  5  4  8]]
Précision: 0.3892914653784219
Rappel: 0.35140288632935696
Score F1: 0.3501440167934587
Accuracy: 0.33


In [96]:
predictions_bge_m3_zeroshot_v2_0_1 = pd.DataFrame({
    'synergy': y_pred_synergy16,
    'antagonism': y_pred_antagonism16
})

### Tache 3 : classification finale

In [97]:
predictions_bge_m3_zeroshot_v2_0_1['class'] =predictions_bge_m3_zeroshot_v2_0_1.apply(assign_group, axis=1)

In [98]:
y_pred16 = predictions_bge_m3_zeroshot_v2_0_1["class"]

In [99]:
cm3 = confusion_matrix(y_test, y_pred16, labels = ["Passif", "Aligné", "Engagé","Moteur","Déchiré", "Opposant", "Irréductible"])
print(cm3)
print("Précision:", precision_score(y_test, y_pred16, average='macro'))
print("Rappel:", recall_score(y_test, y_pred16, average='macro'))
print("Score F1:", f1_score(y_test, y_pred16, average='macro'))
print("Accuracy:", accuracy_score(y_test, y_pred16))

[[ 9  0  5  5  0  8  1]
 [ 0  0  2  3  0  2  0]
 [ 2  0  0  2  0  1  0]
 [ 1  0  1 10  8  2  6]
 [ 1  0  0  2  2  1  0]
 [ 4  0  1  3  1  3  3]
 [ 2  0  0  1  3  2  3]]
Précision: 0.19854581508716845
Rappel: 0.21209029066171922
Score F1: 0.19711709742998856
Accuracy: 0.27


C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Formulation explicite

## Modèle 1 : facebook/bart-large-mnli"

### Tache 1 : classification selon son niveau de synergie

In [100]:
y_pred_synergy21 = []

In [101]:
start = time.time()
for text in avis["Sample_text"]:
    text = translator.translate(text, target_lang='en')
    result = classifier1(text, text_synergy_labels2e, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_synergy21.append("Minimaliste")
    elif result == "2":
        y_pred_synergy21.append("Intéressé")
    elif result == "3" :
        y_pred_synergy21.append("Coopérant")
    else :
        y_pred_synergy21.append("Engagé")
end = time.time()
execution_time211 = end - start
execution_time211 = execution_time211/60

print(f'Temps d\'exécution : {execution_time211:.2f}minutes')


Temps d'exécution : 12.30minutes


In [102]:
y_pred_synergy21 = np.array(y_pred_synergy21)

In [103]:
y_pred_synergy21

array(['Engagé', 'Intéressé', 'Engagé', 'Engagé', 'Engagé', 'Intéressé',
       'Intéressé', 'Engagé', 'Intéressé', 'Intéressé', 'Minimaliste',
       'Intéressé', 'Engagé', 'Engagé', 'Engagé', 'Engagé', 'Engagé',
       'Intéressé', 'Minimaliste', 'Engagé', 'Minimaliste', 'Engagé',
       'Minimaliste', 'Minimaliste', 'Coopérant', 'Engagé', 'Engagé',
       'Engagé', 'Engagé', 'Engagé', 'Engagé', 'Engagé', 'Engagé',
       'Intéressé', 'Coopérant', 'Engagé', 'Engagé', 'Engagé', 'Engagé',
       'Engagé', 'Minimaliste', 'Engagé', 'Engagé', 'Engagé', 'Engagé',
       'Engagé', 'Engagé', 'Engagé', 'Engagé', 'Engagé', 'Intéressé',
       'Engagé', 'Engagé', 'Engagé', 'Engagé', 'Intéressé', 'Intéressé',
       'Engagé', 'Engagé', 'Engagé', 'Engagé', 'Coopérant', 'Engagé',
       'Engagé', 'Engagé', 'Intéressé', 'Intéressé', 'Engagé',
       'Intéressé', 'Minimaliste', 'Engagé', 'Engagé', 'Engagé', 'Engagé',
       'Coopérant', 'Engagé', 'Engagé', 'Intéressé', 'Intéressé',
       'Engagé', 

In [104]:
cm = confusion_matrix(y_test_synergy, y_pred_synergy21, labels = ["Minimaliste", "Intéressé", "Coopérant","Engagé"])
print(cm)
print("Précision:", precision_score(y_test_synergy, y_pred_synergy21, average='macro'))
print("Rappel:", recall_score(y_test_synergy, y_pred_synergy21, average='macro'))
print("Score F1:", f1_score(y_test_synergy, y_pred_synergy21, average='macro'))
print("Accuracy:", accuracy_score(y_test_synergy, y_pred_synergy21))

[[ 8  8  2 12]
 [ 0 12  0 12]
 [ 0  2  3 14]
 [ 0  3  1 23]]
Précision: 0.5892622950819673
Rappel: 0.44410331384015594
Score F1: 0.41839395566839177
Accuracy: 0.46


### Tache 2 : classification selon son niveau d'antagonisme

In [105]:
y_pred_antagonism21 = []

In [106]:
start = time.time()
for text in avis["Sample_text"]:
    text = translator.translate(text, target_lang='en')
    result = classifier1(text, text_antagonism_labels2e, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_antagonism21.append("Conciliant")
    elif result == "2":
        y_pred_antagonism21.append("Résistant")
    elif result == "3" :
        y_pred_antagonism21.append("Opposant")
    else :
        y_pred_antagonism21.append("Irréconciliant")
end = time.time()
execution_time212 = end - start
execution_time212 = execution_time212/60

print(f'Temps d\'exécution : {execution_time212:.2f}minutes')

Temps d'exécution : 23.27minutes


In [107]:
y_pred_antagonism21 = np.array(y_pred_antagonism21)

In [108]:
cm2 = confusion_matrix(y_test_antagonism, y_pred_antagonism21, labels = ["Conciliant", "Résistant", "Opposant","Irréconciliant"])
print(cm2)
print("Précision:", precision_score(y_test_antagonism, y_pred_antagonism21, average='macro'))
print("Rappel:", recall_score(y_test_antagonism, y_pred_antagonism21, average='macro'))
print("Score F1:", f1_score(y_test_antagonism, y_pred_antagonism21, average='macro'))
print("Accuracy:", accuracy_score(y_test_antagonism, y_pred_antagonism21))

[[25  1  0  0]
 [32  0  0  1]
 [15  2  0  7]
 [ 9  0  0  8]]
Précision: 0.2021604938271605
Rappel: 0.3580316742081448
Score F1: 0.23803455111866328
Accuracy: 0.33


C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [109]:
predictions_bart_large_mnli_2 = pd.DataFrame({
    'synergy': y_pred_synergy21,
    'antagonism': y_pred_antagonism21
})

### Tache 3 : classification finale

In [110]:
predictions_bart_large_mnli_2['class'] = predictions_bart_large_mnli_2.apply(assign_group, axis=1)

In [111]:
y_pred21 = predictions_bart_large_mnli_2 ["class"]

In [112]:
cm3 = confusion_matrix(y_test, y_pred21, labels = ["Passif", "Aligné", "Engagé","Moteur","Déchiré", "Opposant", "Irréductible"])
print(cm3)
print("Précision:", precision_score(y_test, y_pred21, average='macro'))
print("Rappel:", recall_score(y_test, y_pred21, average='macro'))
print("Score F1:", f1_score(y_test, y_pred21, average='macro'))
print("Accuracy:", accuracy_score(y_test, y_pred21))

[[15  2 10  0  0  0  1]
 [ 0  2  5  0  0  0  0]
 [ 0  0  4  1  0  0  0]
 [ 5  1 21  1  0  0  0]
 [ 0  0  3  0  3  0  0]
 [ 4  0  4  0  5  0  2]
 [ 4  0  2  0  3  0  2]]
Précision: 0.31286774450039756
Rappel: 0.33413729128014846
Score F1: 0.2409719443332889
Accuracy: 0.27


C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Modèle 2 : "MoritzLaurer/deberta-v3-large-zeroshot-v2.0"

### Tache 1 : classification selon son niveau de synergie

In [113]:
y_pred_synergy22 = []

In [114]:
start = time.time()
for text in avis["Sample_text"]:
    result = classifier2(text, text_synergy_labels2, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_synergy22.append("Minimaliste")
    elif result == "2":
        y_pred_synergy22.append("Intéressé")
    elif result == "3" :
        y_pred_synergy22.append("Coopérant")
    else :
        y_pred_synergy22.append("Engagé")
end = time.time()
execution_time221 = end - start
execution_time221 = execution_time221/60

print(f'Temps d\'exécution : {execution_time221:.2f}minutes')


Temps d'exécution : 29.93minutes


In [115]:
y_pred_synergy22 = np.array(y_pred_synergy22)

In [116]:
cm = confusion_matrix(y_test_synergy, y_pred_synergy22, labels = ["Minimaliste", "Intéressé", "Coopérant","Engagé"])
print(cm)
print("Précision:", precision_score(y_test_synergy, y_pred_synergy22, average='macro'))
print("Rappel:", recall_score(y_test_synergy, y_pred_synergy22, average='macro'))
print("Score F1:", f1_score(y_test_synergy, y_pred_synergy22, average='macro'))
print("Accuracy:", accuracy_score(y_test_synergy, y_pred_synergy22))

[[23  6  0  1]
 [14  5  0  5]
 [ 3  1  8  7]
 [ 0  0  9 18]]
Précision: 0.5107250158127767
Rappel: 0.5156798245614035
Score F1: 0.5000136836343733
Accuracy: 0.54


### Tache 2 : classification selon son niveau d'antagonisme

In [34]:
y_pred_antagonism22 = []

In [35]:
start = time.time()
for text in avis["Sample_text"]:
    result = classifier2(text, text_antagonism_labels2, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_antagonism22.append("Conciliant")
    elif result == "2":
        y_pred_antagonism22.append("Résistant")
    elif result == "3" :
        y_pred_antagonism22.append("Opposant")
    else :
        y_pred_antagonism22.append("Irréconciliant")
end = time.time()
execution_time222 = end - start
execution_time222 = execution_time222/60

print(f'Temps d\'exécution : {execution_time222:.2f}minutes')

Temps d'exécution : 16.94minutes


In [36]:
y_pred_antagonism22 = np.array(y_pred_antagonism22)

In [40]:
cm2 = confusion_matrix(y_test_antagonism, y_pred_antagonism22, labels = ["Conciliant", "Résistant", "Opposant","Irréconciliant"])
print(cm2)
print("Précision:", precision_score(y_test_antagonism, y_pred_antagonism22, average='macro'))
print("Rappel:", recall_score(y_test_antagonism, y_pred_antagonism22, average='macro'))
print("Score F1:", f1_score(y_test_antagonism, y_pred_antagonism22, average='macro'))
print("Accuracy:", accuracy_score(y_test_antagonism, y_pred_antagonism22))

[[25  1  0  0]
 [11 14  8  0]
 [ 7  5  9  3]
 [ 2  0  4 11]]
Précision: 0.6174603174603175
Rappel: 0.6019599273275744
Score F1: 0.5855511645649918
Accuracy: 0.59


In [41]:
predictions_deberta_v3_large_zeroshot_v2_0_2 = pd.DataFrame({
    'synergy': y_pred_synergy22,
    'antagonism': y_pred_antagonism22
})

### Tache 3 : classification finale

In [42]:
predictions_deberta_v3_large_zeroshot_v2_0_2['class'] =predictions_deberta_v3_large_zeroshot_v2_0_2.apply(assign_group, axis=1)

In [43]:
y_pred22 = predictions_deberta_v3_large_zeroshot_v2_0_2["class"]

In [46]:
cm3 = confusion_matrix(y_test, y_pred22, labels = ["Passif", "Aligné", "Engagé","Moteur","Déchiré", "Opposant", "Irréductible"])
print(cm3)
print("Précision:", precision_score(y_test, y_pred22, average='macro'))
print("Rappel:", recall_score(y_test, y_pred22, average='macro'))
print("Score F1:", f1_score(y_test, y_pred22, average='macro'))
print("Accuracy:", accuracy_score(y_test, y_pred22))

[[22  0  3  2  0  1  0]
 [ 2  3  2  0  0  0  0]
 [ 0  2  2  1  0  0  0]
 [ 1  1  4 20  2  0  0]
 [ 0  1  0  1  3  1  0]
 [12  0  0  0  0  2  1]
 [ 1  0  0  0  1  2  7]]
Précision: 0.5330005207824757
Rappel: 0.5140383426097712
Score F1: 0.5059695943154588
Accuracy: 0.59


## Modèle 3 : MoritzLaurer/deberta-v3-large-zeroshot-v2.0-c

## Tache 1 : classification selon son niveau de synergie

In [47]:
y_pred_synergy23 = []

In [48]:
start = time.time()
for text in avis["Sample_text"]:
    result = classifier3(text, text_synergy_labels2, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_synergy23.append("Minimaliste")
    elif result == "2":
        y_pred_synergy23.append("Intéressé")
    elif result == "3" :
        y_pred_synergy23.append("Coopérant")
    else :
        y_pred_synergy23.append("Engagé")
end = time.time()
execution_time231 = end - start
execution_time231 = execution_time231/60

print(f'Temps d\'exécution : {execution_time231:.2f}minutes')

Temps d'exécution : 10.55minutes


In [49]:
y_pred_synergy23 = np.array(y_pred_synergy23)

In [50]:
cm = confusion_matrix(y_test_synergy, y_pred_synergy23, labels = ["Minimaliste", "Intéressé", "Coopérant","Engagé"])
print(cm)
print("Précision:", precision_score(y_test_synergy, y_pred_synergy23, average='macro'))
print("Rappel:", recall_score(y_test_synergy, y_pred_synergy23, average='macro'))
print("Score F1:", f1_score(y_test_synergy, y_pred_synergy23, average='macro'))
print("Accuracy:", accuracy_score(y_test_synergy, y_pred_synergy23))

[[27  2  0  1]
 [19  5  0  0]
 [ 5  3 10  1]
 [ 5  0 16  6]]
Précision: 0.5291895604395604
Rappel: 0.46421783625730995
Score F1: 0.4273315527761492
Accuracy: 0.48


### Tache 2 : classification selon son niveau d'antagonisme

In [51]:
y_pred_antagonism23 = []

In [52]:
start = time.time()
for text in avis["Sample_text"]:
    result = classifier3(text, text_antagonism_labels2, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_antagonism23.append("Conciliant")
    elif result == "2":
        y_pred_antagonism23.append("Résistant")
    elif result == "3" :
        y_pred_antagonism23.append("Opposant")
    else :
        y_pred_antagonism23.append("Irréconciliant")
end = time.time()
execution_time232 = end - start
execution_time232 = execution_time232/60

print(f'Temps d\'exécution : {execution_time232:.2f}minutes')

Temps d'exécution : 11.80minutes


In [53]:
y_pred_antagonism23 = np.array(y_pred_antagonism23)

In [54]:
cm2 = confusion_matrix(y_test_antagonism, y_pred_antagonism23, labels = ["Conciliant", "Résistant", "Opposant","Irréconciliant"])
print(cm2)
print("Précision:", precision_score(y_test_antagonism, y_pred_antagonism23, average='macro'))
print("Rappel:", recall_score(y_test_antagonism, y_pred_antagonism23, average='macro'))
print("Score F1:", f1_score(y_test_antagonism, y_pred_antagonism23, average='macro'))
print("Accuracy:", accuracy_score(y_test_antagonism, y_pred_antagonism23))

[[24  1  0  1]
 [ 7 22  4  0]
 [ 1  8 11  4]
 [ 1  2  2 12]]
Précision: 0.6867201426024955
Rappel: 0.6884898190045249
Score F1: 0.68067342687385
Accuracy: 0.69


In [55]:
predictions_deberta_v3_large_zeroshot_v2_0_c_2 = pd.DataFrame({
    'synergy': y_pred_synergy23,
    'antagonism': y_pred_antagonism23
})

### Tache 3 : classification finale 

In [56]:
predictions_deberta_v3_large_zeroshot_v2_0_c_2['class'] =predictions_deberta_v3_large_zeroshot_v2_0_c_2.apply(assign_group, axis=1)

In [57]:
y_pred23 = predictions_deberta_v3_large_zeroshot_v2_0_c_2["class"]

In [58]:
cm3 = confusion_matrix(y_test, y_pred23, labels = ["Passif", "Aligné", "Engagé","Moteur","Déchiré", "Opposant", "Irréductible"])
print(cm3)
print("Précision:", precision_score(y_test, y_pred23, average='macro'))
print("Rappel:", recall_score(y_test, y_pred23, average='macro'))
print("Score F1:", f1_score(y_test, y_pred23, average='macro'))
print("Accuracy:", accuracy_score(y_test, y_pred23))

[[26  0  0  0  0  1  1]
 [ 3  4  0  0  0  0  0]
 [ 0  4  0  1  0  0  0]
 [ 8  3  0 13  3  1  0]
 [ 0  0  0  2  3  0  1]
 [ 7  0  0  0  0  7  1]
 [ 2  0  0  0  1  1  7]]
Précision: 0.5099893119303057
Rappel: 0.5096165739022883
Score F1: 0.4894659094659094
Accuracy: 0.6


C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Modèle 4 : MoritzLaurer/deberta-v3-base-zeroshot-v2.0

### Tache 1 : classification selon son niveau de synergie

In [59]:
y_pred_synergy24 = []

In [60]:
start = time.time()
for text in avis["Sample_text"]:
    result = classifier4(text, text_synergy_labels2, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_synergy24.append("Minimaliste")
    elif result == "2":
        y_pred_synergy24.append("Intéressé")
    elif result == "3" :
        y_pred_synergy24.append("Coopérant")
    else :
        y_pred_synergy24.append("Engagé")
end = time.time()
execution_time241 = end - start
execution_time241 = execution_time241/60

print(f'Temps d\'exécution : {execution_time241:.2f}minutes')

Temps d'exécution : 3.39minutes


In [61]:
y_pred_synergy24 = np.array(y_pred_synergy24 )

In [62]:
cm = confusion_matrix(y_test_synergy, y_pred_synergy24, labels = ["Minimaliste", "Intéressé", "Coopérant","Engagé"])
print(cm)
print("Précision:", precision_score(y_test_synergy, y_pred_synergy24, average='macro'))
print("Rappel:", recall_score(y_test_synergy, y_pred_synergy24, average='macro'))
print("Score F1:", f1_score(y_test_synergy, y_pred_synergy24, average='macro'))
print("Accuracy:", accuracy_score(y_test_synergy, y_pred_synergy24))

[[17  5  6  2]
 [ 5  3 11  5]
 [ 0  0 14  5]
 [ 0  0 18  9]]
Précision: 0.4655032467532467
Rappel: 0.44046052631578947
Score F1: 0.4070277149321267
Accuracy: 0.43


### Tache 2 : classification selon son niveau d'antagonisme

In [63]:
y_pred_antagonism24 = []

In [64]:
start = time.time()
for text in avis["Sample_text"]:
    result = classifier4(text, text_antagonism_labels2, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_antagonism24.append("Conciliant")
    elif result == "2":
        y_pred_antagonism24.append("Résistant")
    elif result == "3" :
        y_pred_antagonism24.append("Opposant")
    else :
        y_pred_antagonism24.append("Irréconciliant")
end = time.time()
execution_time242 = end - start
execution_time242 = execution_time242/60

print(f'Temps d\'exécution : {execution_time242:.2f}minutes')

Temps d'exécution : 3.44minutes


In [65]:
y_pred_antagonism24 = np.array(y_pred_antagonism24)

In [66]:
cm2 = confusion_matrix(y_test_antagonism, y_pred_antagonism24, labels = ["Conciliant", "Résistant", "Opposant","Irréconciliant"])
print(cm2)
print("Précision:", precision_score(y_test_antagonism, y_pred_antagonism24, average='macro'))
print("Rappel:", recall_score(y_test_antagonism, y_pred_antagonism24, average='macro'))
print("Score F1:", f1_score(y_test_antagonism, y_pred_antagonism24, average='macro'))
print("Accuracy:", accuracy_score(y_test_antagonism, y_pred_antagonism24))

[[20  0  4  2]
 [13  5  4 11]
 [ 5  2  3 14]
 [ 1  0  1 15]]
Précision: 0.4585622710622711
Rappel: 0.48202471548059783
Score F1: 0.38513146458061714
Accuracy: 0.43


In [67]:
predictions_deberta_v3_base_zeroshot_v2_0_2 = pd.DataFrame({
    'synergy': y_pred_synergy24,
    'antagonism': y_pred_antagonism24
})

### Tache 3 : classification finale

In [68]:
predictions_deberta_v3_base_zeroshot_v2_0_2['class'] =predictions_deberta_v3_base_zeroshot_v2_0_2.apply(assign_group, axis=1)

In [69]:
y_pred24 = predictions_deberta_v3_base_zeroshot_v2_0_2["class"]

In [70]:
cm3 = confusion_matrix(y_test, y_pred24, labels = ["Passif", "Aligné", "Engagé","Moteur","Déchiré", "Opposant", "Irréductible"])
print(cm3)
print("Précision:", precision_score(y_test, y_pred24, average='macro'))
print("Rappel:", recall_score(y_test, y_pred24, average='macro'))
print("Score F1:", f1_score(y_test, y_pred24, average='macro'))
print("Accuracy:", accuracy_score(y_test, y_pred24))

[[ 8  5  4  1  4  3  3]
 [ 0  2  3  1  1  0  0]
 [ 0  4  1  0  0  0  0]
 [ 0  7  0 11 10  0  0]
 [ 0  0  0  1  5  0  0]
 [ 2  2  0  2  3  0  6]
 [ 1  0  0  0  3  0  7]]
Précision: 0.32422577422577425
Rappel: 0.37628324056895485
Score F1: 0.2918956043956044
Accuracy: 0.34


## Modèle 5 : MoritzLaurer/roberta-large-zeroshot-v2.0

### Tache 1 : classification selon son niveau de synergie

In [71]:
y_pred_synergy25 = []

In [72]:
start = time.time()
for text in avis["Sample_text"]:
    text = translator.translate(text, target_lang='en')
    result = classifier5(text, text_synergy_labels2e, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_synergy25.append("Minimaliste")
    elif result == "2":
        y_pred_synergy25.append("Intéressé")
    elif result == "3" :
        y_pred_synergy25.append("Coopérant")
    else :
        y_pred_synergy25.append("Engagé")
end = time.time()
execution_time251 = end - start
execution_time251 = execution_time251/60

print(f'Temps d\'exécution : {execution_time251:.2f}minutes')


C:\Users\HP\anaconda3\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Temps d'exécution : 10.92minutes


In [73]:
y_pred_synergy25 = np.array(y_pred_synergy25 )

In [74]:
cm = confusion_matrix(y_test_synergy, y_pred_synergy25, labels = ["Minimaliste", "Intéressé", "Coopérant","Engagé"])
print(cm)
print("Précision:", precision_score(y_test_synergy, y_pred_synergy25, average='macro'))
print("Rappel:", recall_score(y_test_synergy, y_pred_synergy25, average='macro'))
print("Score F1:", f1_score(y_test_synergy, y_pred_synergy25, average='macro'))
print("Accuracy:", accuracy_score(y_test_synergy, y_pred_synergy25))

[[29  1  0  0]
 [21  3  0  0]
 [ 4  3 12  0]
 [12  0 13  2]]
Précision: 0.586991341991342
Rappel: 0.44932992202729044
Score F1: 0.3702751584251862
Accuracy: 0.46


### Tache 2 : classification selon son niveau d'antagonisme

In [75]:
y_pred_antagonism25 = []

In [76]:
start = time.time()
for text in avis["Sample_text"]:
    text = translator.translate(text, target_lang='en')
    result = classifier5(text, text_antagonism_labels2e, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_antagonism25.append("Conciliant")
    elif result == "2":
        y_pred_antagonism25.append("Résistant")
    elif result == "3" :
        y_pred_antagonism25.append("Opposant")
    else :
        y_pred_antagonism25.append("Irréconciliant")
end = time.time()
execution_time252 = end - start
execution_time252 = execution_time252/60

print(f'Temps d\'exécution : {execution_time252:.2f}minutes')

Temps d'exécution : 10.76minutes


In [77]:
y_pred_antagonism25 = np.array(y_pred_antagonism25)

In [78]:
cm2 = confusion_matrix(y_test_antagonism, y_pred_antagonism25, labels = ["Conciliant", "Résistant", "Opposant","Irréconciliant"])
print(cm2)
print("Précision:", precision_score(y_test_antagonism, y_pred_antagonism25, average='macro'))
print("Rappel:", recall_score(y_test_antagonism, y_pred_antagonism25, average='macro'))
print("Score F1:", f1_score(y_test_antagonism, y_pred_antagonism25, average='macro'))
print("Accuracy:", accuracy_score(y_test_antagonism, y_pred_antagonism25))

[[26  0  0  0]
 [ 6 25  1  1]
 [ 2 13  5  4]
 [ 3  3  0 11]]
Précision: 0.708323033399253
Rappel: 0.6532419786096256
Score F1: 0.6252681252681254
Accuracy: 0.67


In [79]:
predictions_roberta_large_zeroshot_v2_0_2 = pd.DataFrame({
    'synergy': y_pred_synergy25,
    'antagonism': y_pred_antagonism25
})

### Tache 3 : classification finale

In [80]:
predictions_roberta_large_zeroshot_v2_0_2['class'] =predictions_roberta_large_zeroshot_v2_0_2.apply(assign_group, axis=1)

In [81]:
y_pred25 = predictions_roberta_large_zeroshot_v2_0_2["class"]

In [82]:
cm3 = confusion_matrix(y_test, y_pred25, labels = ["Passif", "Aligné", "Engagé","Moteur","Déchiré", "Opposant", "Irréductible"])
print(cm3)
print("Précision:", precision_score(y_test, y_pred25, average='macro'))
print("Rappel:", recall_score(y_test, y_pred25, average='macro'))
print("Score F1:", f1_score(y_test, y_pred25, average='macro'))
print("Accuracy:", accuracy_score(y_test, y_pred25))

[[27  0  0  0  0  1  0]
 [ 2  5  0  0  0  0  0]
 [ 0  4  1  0  0  0  0]
 [11  4  0  8  2  1  2]
 [ 1  0  0  1  2  0  2]
 [11  0  0  0  0  3  1]
 [ 4  0  0  0  0  0  7]]
Précision: 0.6341400662829234
Rappel: 0.4762832405689549
Score F1: 0.4596169372566888
Accuracy: 0.53


## Modèle 6 : MoritzLaurer/bge-m3-zeroshot-v2.0

### Tache 1 : classification selon son niveau de synergie

In [83]:
y_pred_synergy26 = []

In [84]:
start = time.time()
for text in avis["Sample_text"]:
    result = classifier6(text, text_synergy_labels2, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_synergy26.append("Minimaliste")
    elif result == "2":
        y_pred_synergy26.append("Intéressé")
    elif result == "3" :
        y_pred_synergy26.append("Coopérant")
    else :
        y_pred_synergy26.append("Engagé")
end = time.time()
execution_time261 = end - start
execution_time261 = execution_time261/60

print(f'Temps d\'exécution : {execution_time261:.2f}minutes')

Temps d'exécution : 5.96minutes


In [85]:
y_pred_synergy26 = np.array(y_pred_synergy26 )

In [86]:
cm = confusion_matrix(y_test_synergy, y_pred_synergy26, labels = ["Minimaliste", "Intéressé", "Coopérant","Engagé"])
print(cm)
print("Précision:", precision_score(y_test_synergy, y_pred_synergy26, average='macro'))
print("Rappel:", recall_score(y_test_synergy, y_pred_synergy26, average='macro'))
print("Score F1:", f1_score(y_test_synergy, y_pred_synergy26, average='macro'))
print("Accuracy:", accuracy_score(y_test_synergy, y_pred_synergy26))

[[29  1  0  0]
 [19  2  2  1]
 [ 3  0 13  3]
 [ 3  0 14 10]]
Précision: 0.5915663200145959
Rappel: 0.52614522417154
Score F1: 0.4670239708349464
Accuracy: 0.54


### Tache 2 : classification selon son niveau d'antagonisme

In [87]:
y_pred_antagonism26 = []

In [88]:
start = time.time()
for text in avis["Sample_text"]:
    result = classifier6(text, text_antagonism_labels2, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_antagonism26.append("Conciliant")
    elif result == "2":
        y_pred_antagonism26.append("Résistant")
    elif result == "3" :
        y_pred_antagonism26.append("Opposant")
    else :
        y_pred_antagonism26.append("Irréconciliant")
end = time.time()
execution_time262 = end - start
execution_time262 = execution_time262/60

print(f'Temps d\'exécution : {execution_time262:.2f}minutes')

Temps d'exécution : 5.66minutes


In [89]:
y_pred_antagonism26 = np.array(y_pred_antagonism26)

In [90]:
cm2 = confusion_matrix(y_test_antagonism, y_pred_antagonism26, labels = ["Conciliant", "Résistant", "Opposant","Irréconciliant"])
print(cm2)
print("Précision:", precision_score(y_test_antagonism, y_pred_antagonism26, average='macro'))
print("Rappel:", recall_score(y_test_antagonism, y_pred_antagonism26, average='macro'))
print("Score F1:", f1_score(y_test_antagonism, y_pred_antagonism26, average='macro'))
print("Accuracy:", accuracy_score(y_test_antagonism, y_pred_antagonism26))

[[23  0  2  1]
 [15 10  7  1]
 [ 9  9  3  3]
 [ 3  2  1 11]]
Précision: 0.46361492673992677
Rappel: 0.4899261277937748
Score F1: 0.45111558927348405
Accuracy: 0.47


In [91]:
predictions_bge_m3_zeroshot_v2_0_2 = pd.DataFrame({
    'synergy': y_pred_synergy26,
    'antagonism': y_pred_antagonism26
})

### Tache 3 : classification finale

In [92]:
predictions_bge_m3_zeroshot_v2_0_2['class'] =predictions_bge_m3_zeroshot_v2_0_2.apply(assign_group, axis=1)

In [93]:
y_pred26 = predictions_bge_m3_zeroshot_v2_0_2["class"]

In [94]:
cm3 = confusion_matrix(y_test, y_pred26, labels = ["Passif", "Aligné", "Engagé","Moteur","Déchiré", "Opposant", "Irréductible"])
print(cm3)
print("Précision:", precision_score(y_test, y_pred26, average='macro'))
print("Rappel:", recall_score(y_test, y_pred26, average='macro'))
print("Score F1:", f1_score(y_test, y_pred26, average='macro'))
print("Accuracy:", accuracy_score(y_test, y_pred26))

[[19  2  1  0  0  5  1]
 [ 1  4  1  0  1  0  0]
 [ 0  2  2  1  0  0  0]
 [ 3  6  4 13  1  0  1]
 [ 0  1  0  1  3  0  1]
 [12  0  0  0  0  2  1]
 [ 4  0  0  0  0  0  7]]
Précision: 0.48465582037010607
Rappel: 0.48342609771181205
Score F1: 0.45811148240860133
Accuracy: 0.5


# Formulation optimale

## Modèle 1 : facebook/bart-large-mnli"

### Tache 1 : classification selon son niveau de synergie

In [95]:
y_pred_synergy31 = []

In [96]:
start = time.time()
for text in avis["Sample_text"]:
    text = translator.translate(text, target_lang='en')
    result = classifier1(text, text_synergy_labels3e, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_synergy31.append("Minimaliste")
    elif result == "2":
        y_pred_synergy31.append("Intéressé")
    elif result == "3" :
        y_pred_synergy31.append("Coopérant")
    else :
        y_pred_synergy31.append("Engagé")
end = time.time()
execution_time311 = end - start
execution_time311 = execution_time311/60

print(f'Temps d\'exécution : {execution_time311:.2f}minutes')


Temps d'exécution : 11.53minutes


In [97]:
y_pred_synergy31 = np.array(y_pred_synergy31)

In [98]:
y_pred_synergy31

array(['Intéressé', 'Intéressé', 'Engagé', 'Intéressé', 'Intéressé',
       'Minimaliste', 'Intéressé', 'Minimaliste', 'Intéressé',
       'Minimaliste', 'Intéressé', 'Coopérant', 'Intéressé', 'Engagé',
       'Minimaliste', 'Engagé', 'Intéressé', 'Intéressé', 'Intéressé',
       'Intéressé', 'Minimaliste', 'Intéressé', 'Minimaliste',
       'Minimaliste', 'Minimaliste', 'Intéressé', 'Intéressé',
       'Intéressé', 'Intéressé', 'Engagé', 'Intéressé', 'Intéressé',
       'Intéressé', 'Intéressé', 'Intéressé', 'Intéressé', 'Engagé',
       'Minimaliste', 'Intéressé', 'Engagé', 'Minimaliste', 'Intéressé',
       'Intéressé', 'Intéressé', 'Intéressé', 'Intéressé', 'Intéressé',
       'Intéressé', 'Intéressé', 'Coopérant', 'Intéressé', 'Intéressé',
       'Minimaliste', 'Intéressé', 'Intéressé', 'Intéressé', 'Intéressé',
       'Engagé', 'Intéressé', 'Engagé', 'Coopérant', 'Engagé',
       'Intéressé', 'Coopérant', 'Coopérant', 'Intéressé', 'Minimaliste',
       'Intéressé', 'Intéressé', '

In [99]:
cm = confusion_matrix(y_test_synergy, y_pred_synergy31, labels = ["Minimaliste", "Intéressé", "Coopérant","Engagé"])
print(cm)
print("Précision:", precision_score(y_test_synergy, y_pred_synergy31, average='macro'))
print("Rappel:", recall_score(y_test_synergy, y_pred_synergy31, average='macro'))
print("Score F1:", f1_score(y_test_synergy, y_pred_synergy31, average='macro'))
print("Accuracy:", accuracy_score(y_test_synergy, y_pred_synergy31))

[[13 17  0  0]
 [ 2 21  0  1]
 [ 2  7  6  4]
 [ 1 14  6  6]]
Précision: 0.5309022427666495
Rappel: 0.4615862573099415
Score F1: 0.43764425273249197
Accuracy: 0.46


### Tache 2 : classification selon son niveau d'antagonisme

In [100]:
y_pred_antagonism31 = []

In [101]:
start = time.time()
for text in avis["Sample_text"]:
    text = translator.translate(text, target_lang='en')
    result = classifier1(text, text_antagonism_labels3e, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_antagonism31.append("Conciliant")
    elif result == "2":
        y_pred_antagonism31.append("Résistant")
    elif result == "3" :
        y_pred_antagonism31.append("Opposant")
    else :
        y_pred_antagonism31.append("Irréconciliant")
end = time.time()
execution_time312 = end - start
execution_time312 = execution_time312/60

print(f'Temps d\'exécution : {execution_time312:.2f}minutes')

Temps d'exécution : 10.80minutes


In [102]:
y_pred_antagonism31 = np.array(y_pred_antagonism31)

In [103]:
cm2 = confusion_matrix(y_test_antagonism, y_pred_antagonism31, labels = ["Conciliant", "Résistant", "Opposant","Irréconciliant"])
print(cm2)
print("Précision:", precision_score(y_test_antagonism, y_pred_antagonism31, average='macro'))
print("Rappel:", recall_score(y_test_antagonism, y_pred_antagonism31, average='macro'))
print("Score F1:", f1_score(y_test_antagonism, y_pred_antagonism31, average='macro'))
print("Accuracy:", accuracy_score(y_test_antagonism, y_pred_antagonism31))

[[26  0  0  0]
 [ 5 28  0  0]
 [ 6 16  2  0]
 [ 3  5  7  2]]
Précision: 0.6109126984126985
Rappel: 0.5123663101604278
Score F1: 0.45063601353716887
Accuracy: 0.58


In [104]:
predictions_bart_large_mnli_3 = pd.DataFrame({
    'synergy': y_pred_synergy31,
    'antagonism': y_pred_antagonism31
})

### Tache 3 : classification finale

In [105]:
predictions_bart_large_mnli_3['class'] = predictions_bart_large_mnli_3.apply(assign_group, axis=1)

In [106]:
y_pred31 = predictions_bart_large_mnli_3 ["class"]

In [107]:
cm3 = confusion_matrix(y_test, y_pred31, labels = ["Passif", "Aligné", "Engagé","Moteur","Déchiré", "Opposant", "Irréductible"])
print(cm3)
print("Précision:", precision_score(y_test, y_pred31, average='macro'))
print("Rappel:", recall_score(y_test, y_pred31, average='macro'))
print("Score F1:", f1_score(y_test, y_pred31, average='macro'))
print("Accuracy:", accuracy_score(y_test, y_pred31))

[[27  0  0  1  0  0  0]
 [ 3  2  2  0  0  0  0]
 [ 2  2  1  0  0  0  0]
 [16  1  2  9  0  0  0]
 [ 2  0  0  3  0  1  0]
 [13  0  0  0  0  2  0]
 [ 6  0  0  0  0  3  2]]
Précision: 0.4309921962095875
Rappel: 0.298082869511441
Score F1: 0.2896038932390813
Accuracy: 0.43


C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Modèle 2 : "MoritzLaurer/deberta-v3-large-zeroshot-v2.0"

### Tache 1 : classification selon son niveau de synergie

In [108]:
y_pred_synergy32 = []

In [109]:
start = time.time()
for text in avis["Sample_text"]:
    result = classifier2(text, text_synergy_labels3, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_synergy32.append("Minimaliste")
    elif result == "2":
        y_pred_synergy32.append("Intéressé")
    elif result == "3" :
        y_pred_synergy32.append("Coopérant")
    else :
        y_pred_synergy32.append("Engagé")
end = time.time()
execution_time321 = end - start
execution_time321 = execution_time321/60

print(f'Temps d\'exécution : {execution_time321:.2f}minutes')


Temps d'exécution : 10.42minutes


In [110]:
y_pred_synergy32 = np.array(y_pred_synergy32)

In [111]:
cm = confusion_matrix(y_test_synergy, y_pred_synergy32, labels = ["Minimaliste", "Intéressé", "Coopérant","Engagé"])
print(cm)
print("Précision:", precision_score(y_test_synergy, y_pred_synergy32, average='macro'))
print("Rappel:", recall_score(y_test_synergy, y_pred_synergy32, average='macro'))
print("Score F1:", f1_score(y_test_synergy, y_pred_synergy32, average='macro'))
print("Accuracy:", accuracy_score(y_test_synergy, y_pred_synergy32))

[[26  4  0  0]
 [ 3 19  1  1]
 [ 1  0 13  5]
 [ 0  0  8 19]]
Précision: 0.7609156785243743
Rappel: 0.7615618908382066
Score F1: 0.7600232192992962
Accuracy: 0.77


### Tache 2 : classification selon son niveau d'antagonisme

In [112]:
y_pred_antagonism32 = []

In [113]:
start = time.time()
for text in avis["Sample_text"]:
    result = classifier2(text, text_antagonism_labels3, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_antagonism32.append("Conciliant")
    elif result == "2":
        y_pred_antagonism32.append("Résistant")
    elif result == "3" :
        y_pred_antagonism32.append("Opposant")
    else :
        y_pred_antagonism32.append("Irréconciliant")
end = time.time()
execution_time322 = end - start
execution_time322 = execution_time322/60

print(f'Temps d\'exécution : {execution_time322:.2f}minutes')

Temps d'exécution : 10.80minutes


In [114]:
y_pred_antagonism32 = np.array(y_pred_antagonism32)

In [115]:
cm2 = confusion_matrix(y_test_antagonism, y_pred_antagonism32, labels = ["Conciliant", "Résistant", "Opposant","Irréconciliant"])
print(cm2)
print("Précision:", precision_score(y_test_antagonism, y_pred_antagonism32, average='macro'))
print("Rappel:", recall_score(y_test_antagonism, y_pred_antagonism32, average='macro'))
print("Score F1:", f1_score(y_test_antagonism, y_pred_antagonism32, average='macro'))
print("Accuracy:", accuracy_score(y_test_antagonism, y_pred_antagonism32))

[[25  1  0  0]
 [ 4 29  0  0]
 [ 0  7 16  1]
 [ 0  3  3 11]]
Précision: 0.8364602238354507
Rappel: 0.7885129576306047
Score F1: 0.8016045483007287
Accuracy: 0.81


In [116]:
predictions_deberta_v3_large_zeroshot_v2_0_3 = pd.DataFrame({
    'synergy': y_pred_synergy32,
    'antagonism': y_pred_antagonism32
})

### Tache 3 : classification finale

In [117]:
predictions_deberta_v3_large_zeroshot_v2_0_3['class'] =predictions_deberta_v3_large_zeroshot_v2_0_3.apply(assign_group, axis=1)

In [118]:
y_pred32 = predictions_deberta_v3_large_zeroshot_v2_0_3["class"]

In [119]:
cm3 = confusion_matrix(y_test, y_pred32, labels = ["Passif", "Aligné", "Engagé","Moteur","Déchiré", "Opposant", "Irréductible"])
print(cm3)
print("Précision:", precision_score(y_test, y_pred32, average='macro'))
print("Rappel:", recall_score(y_test, y_pred32, average='macro'))
print("Score F1:", f1_score(y_test, y_pred32, average='macro'))
print("Accuracy:", accuracy_score(y_test, y_pred32))

[[26  1  0  1  0  0  0]
 [ 0  5  2  0  0  0  0]
 [ 0  3  2  0  0  0  0]
 [ 1  2  0 24  1  0  0]
 [ 0  0  0  2  4  0  0]
 [ 3  0  0  0  0 12  0]
 [ 2  0  0  0  0  2  7]]
Précision: 0.7590110286538858
Rappel: 0.7147186147186148
Score F1: 0.7245758073344281
Accuracy: 0.8


## Modèle 3 : MoritzLaurer/deberta-v3-large-zeroshot-v2.0-c

## Tache 1 : classification selon son niveau de synergie

In [120]:
y_pred_synergy33 = []

In [121]:
start = time.time()
for text in avis["Sample_text"]:
    result = classifier3(text, text_synergy_labels3, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_synergy33.append("Minimaliste")
    elif result == "2":
        y_pred_synergy33.append("Intéressé")
    elif result == "3" :
        y_pred_synergy33.append("Coopérant")
    else :
        y_pred_synergy33.append("Engagé")
end = time.time()
execution_time331 = end - start
execution_time331 = execution_time231/60

print(f'Temps d\'exécution : {execution_time331:.2f}minutes')

Temps d'exécution : 0.18minutes


In [122]:
y_pred_synergy33 = np.array(y_pred_synergy33)

In [123]:
cm = confusion_matrix(y_test_synergy, y_pred_synergy33, labels = ["Minimaliste", "Intéressé", "Coopérant","Engagé"])
print(cm)
print("Précision:", precision_score(y_test_synergy, y_pred_synergy33, average='macro'))
print("Rappel:", recall_score(y_test_synergy, y_pred_synergy33, average='macro'))
print("Score F1:", f1_score(y_test_synergy, y_pred_synergy33, average='macro'))
print("Accuracy:", accuracy_score(y_test_synergy, y_pred_synergy33))

[[25  2  1  2]
 [ 6 15  2  1]
 [ 1  0 15  3]
 [ 0  1 18  8]]
Précision: 0.6506696428571429
Rappel: 0.636025828460039
Score F1: 0.6141089437706275
Accuracy: 0.63


### Tache 2 : classification selon son niveau d'antagonisme

In [124]:
y_pred_antagonism33 = []

In [125]:
start = time.time()
for text in avis["Sample_text"]:
    result = classifier3(text, text_antagonism_labels3, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_antagonism33.append("Conciliant")
    elif result == "2":
        y_pred_antagonism33.append("Résistant")
    elif result == "3" :
        y_pred_antagonism33.append("Opposant")
    else :
        y_pred_antagonism33.append("Irréconciliant")
end = time.time()
execution_time332 = end - start
execution_time332 = execution_time332/60

print(f'Temps d\'exécution : {execution_time332:.2f}minutes')

Temps d'exécution : 10.65minutes


In [126]:
y_pred_antagonism33 = np.array(y_pred_antagonism33)

In [127]:
cm2 = confusion_matrix(y_test_antagonism, y_pred_antagonism33, labels = ["Conciliant", "Résistant", "Opposant","Irréconciliant"])
print(cm2)
print("Précision:", precision_score(y_test_antagonism, y_pred_antagonism33, average='macro'))
print("Rappel:", recall_score(y_test_antagonism, y_pred_antagonism33, average='macro'))
print("Score F1:", f1_score(y_test_antagonism, y_pred_antagonism33, average='macro'))
print("Accuracy:", accuracy_score(y_test_antagonism, y_pred_antagonism33))

[[25  1  0  0]
 [ 4 29  0  0]
 [ 0 11 12  1]
 [ 0  2  4 11]]
Précision: 0.8007885592087677
Rappel: 0.7468462909639381
Score F1: 0.7577173733707309
Accuracy: 0.77


In [128]:
predictions_deberta_v3_large_zeroshot_v2_0_c_3 = pd.DataFrame({
    'synergy': y_pred_synergy33,
    'antagonism': y_pred_antagonism33
})

### Tache 3 : classification finale 

In [129]:
predictions_deberta_v3_large_zeroshot_v2_0_c_3['class'] =predictions_deberta_v3_large_zeroshot_v2_0_c_3.apply(assign_group, axis=1)

In [130]:
y_pred33 = predictions_deberta_v3_large_zeroshot_v2_0_c_3["class"]

In [131]:
cm3 = confusion_matrix(y_test, y_pred33, labels = ["Passif", "Aligné", "Engagé","Moteur","Déchiré", "Opposant", "Irréductible"])
print(cm3)
print("Précision:", precision_score(y_test, y_pred33, average='macro'))
print("Rappel:", recall_score(y_test, y_pred33, average='macro'))
print("Score F1:", f1_score(y_test, y_pred33, average='macro'))
print("Accuracy:", accuracy_score(y_test, y_pred33))

[[26  1  0  1  0  0  0]
 [ 0  6  1  0  0  0  0]
 [ 0  5  0  0  0  0  0]
 [ 2  3  0 22  1  0  0]
 [ 0  0  0  2  4  0  0]
 [ 5  0  0  2  0  8  0]
 [ 1  0  0  0  2  3  5]]
Précision: 0.6111745708384364
Rappel: 0.6037105751391465
Score F1: 0.5771333505204472
Accuracy: 0.71


## Modèle 4 : MoritzLaurer/deberta-v3-base-zeroshot-v2.0

### Tache 1 : classification selon son niveau de synergie

In [132]:
y_pred_synergy34 = []

In [133]:
start = time.time()
for text in avis["Sample_text"]:
    result = classifier4(text, text_synergy_labels3, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_synergy34.append("Minimaliste")
    elif result == "2":
        y_pred_synergy34.append("Intéressé")
    elif result == "3" :
        y_pred_synergy34.append("Coopérant")
    else :
        y_pred_synergy34.append("Engagé")
end = time.time()
execution_time341 = end - start
execution_time341 = execution_time341/60

print(f'Temps d\'exécution : {execution_time341:.2f}minutes')

Temps d'exécution : 2.88minutes


In [134]:
y_pred_synergy34 = np.array(y_pred_synergy34 )

In [135]:
cm = confusion_matrix(y_test_synergy, y_pred_synergy34, labels = ["Minimaliste", "Intéressé", "Coopérant","Engagé"])
print(cm)
print("Précision:", precision_score(y_test_synergy, y_pred_synergy34, average='macro'))
print("Rappel:", recall_score(y_test_synergy, y_pred_synergy34, average='macro'))
print("Score F1:", f1_score(y_test_synergy, y_pred_synergy34, average='macro'))
print("Accuracy:", accuracy_score(y_test_synergy, y_pred_synergy34))

[[26  1  0  3]
 [13  4  3  4]
 [ 1  0  5 13]
 [ 4  1  8 14]]
Précision: 0.49546011586452765
Rappel: 0.45375243664717346
Score F1: 0.4285250121315695
Accuracy: 0.49


### Tache 2 : classification selon son niveau d'antagonisme

In [136]:
y_pred_antagonism34 = []

In [137]:
start = time.time()
for text in avis["Sample_text"]:
    result = classifier4(text, text_antagonism_labels3, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_antagonism34.append("Conciliant")
    elif result == "2":
        y_pred_antagonism34.append("Résistant")
    elif result == "3" :
        y_pred_antagonism34.append("Opposant")
    else :
        y_pred_antagonism34.append("Irréconciliant")
end = time.time()
execution_time342 = end - start
execution_time342 = execution_time342/60

print(f'Temps d\'exécution : {execution_time342:.2f}minutes')

Temps d'exécution : 2.96minutes


In [138]:
y_pred_antagonism34 = np.array(y_pred_antagonism34)

In [139]:
cm2 = confusion_matrix(y_test_antagonism, y_pred_antagonism34, labels = ["Conciliant", "Résistant", "Opposant","Irréconciliant"])
print(cm2)
print("Précision:", precision_score(y_test_antagonism, y_pred_antagonism34, average='macro'))
print("Rappel:", recall_score(y_test_antagonism, y_pred_antagonism34, average='macro'))
print("Score F1:", f1_score(y_test_antagonism, y_pred_antagonism34, average='macro'))
print("Accuracy:", accuracy_score(y_test_antagonism, y_pred_antagonism34))

[[26  0  0  0]
 [20 10  3  0]
 [13  3  7  1]
 [ 4  0  3 10]]
Précision: 0.6573704073704073
Rappel: 0.5457330659536542
Score F1: 0.5279290910702733
Accuracy: 0.53


In [141]:
predictions_deberta_v3_base_zeroshot_v2_0_3 = pd.DataFrame({
    'synergy': y_pred_synergy34,
    'antagonism': y_pred_antagonism34
})

### Tache 3 : classification finale

In [142]:
predictions_deberta_v3_base_zeroshot_v2_0_3['class'] =predictions_deberta_v3_base_zeroshot_v2_0_3.apply(assign_group, axis=1)

In [143]:
y_pred34 = predictions_deberta_v3_base_zeroshot_v2_0_3["class"]

In [144]:
cm3 = confusion_matrix(y_test, y_pred34, labels = ["Passif", "Aligné", "Engagé","Moteur","Déchiré", "Opposant", "Irréductible"])
print(cm3)
print("Précision:", precision_score(y_test, y_pred34, average='macro'))
print("Rappel:", recall_score(y_test, y_pred34, average='macro'))
print("Score F1:", f1_score(y_test, y_pred34, average='macro'))
print("Accuracy:", accuracy_score(y_test, y_pred34))

[[18  0  5  3  0  2  0]
 [ 0  2  5  0  0  0  0]
 [ 0  1  4  0  0  0  0]
 [ 5  3 13  6  1  0  0]
 [ 0  2  1  0  2  0  1]
 [ 9  1  0  1  0  4  0]
 [ 1  0  0  0  0  3  7]]
Précision: 0.4995207173778602
Rappel: 0.4541743970315399
Score F1: 0.41614250479651693
Accuracy: 0.43


## Modèle 5 : MoritzLaurer/roberta-large-zeroshot-v2.0

### Tache 1 : classification selon son niveau de synergie

In [145]:
y_pred_synergy35 = []

In [146]:
start = time.time()
for text in avis["Sample_text"]:
    text = translator.translate(text, target_lang='en')
    result = classifier5(text, text_synergy_labels3e, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_synergy35.append("Minimaliste")
    elif result == "2":
        y_pred_synergy35.append("Intéressé")
    elif result == "3" :
        y_pred_synergy35.append("Coopérant")
    else :
        y_pred_synergy35.append("Engagé")
end = time.time()
execution_time351 = end - start
execution_time351 = execution_time351/60

print(f'Temps d\'exécution : {execution_time351:.2f}minutes')


Temps d'exécution : 9.78minutes


In [147]:
y_pred_synergy35 = np.array(y_pred_synergy35 )

In [148]:
cm = confusion_matrix(y_test_synergy, y_pred_synergy35, labels = ["Minimaliste", "Intéressé", "Coopérant","Engagé"])
print(cm)
print("Précision:", precision_score(y_test_synergy, y_pred_synergy35, average='macro'))
print("Rappel:", recall_score(y_test_synergy, y_pred_synergy35, average='macro'))
print("Score F1:", f1_score(y_test_synergy, y_pred_synergy35, average='macro'))
print("Accuracy:", accuracy_score(y_test_synergy, y_pred_synergy35))

[[26  2  0  2]
 [ 7 16  1  0]
 [ 0  7  7  5]
 [ 2  6  4 15]]
Précision: 0.6310344225666806
Rappel: 0.6143274853801169
Score F1: 0.611418995750793
Accuracy: 0.64


### Tache 2 : classification selon son niveau d'antagonisme

In [149]:
y_pred_antagonism35 = []

In [150]:
start = time.time()
for text in avis["Sample_text"]:
    text = translator.translate(text, target_lang='en')
    result = classifier5(text, text_antagonism_labels3e, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_antagonism35.append("Conciliant")
    elif result == "2":
        y_pred_antagonism35.append("Résistant")
    elif result == "3" :
        y_pred_antagonism35.append("Opposant")
    else :
        y_pred_antagonism35.append("Irréconciliant")
end = time.time()
execution_time352 = end - start
execution_time352 = execution_time352/60

print(f'Temps d\'exécution : {execution_time352:.2f}minutes')

Temps d'exécution : 10.41minutes


In [151]:
y_pred_antagonism35 = np.array(y_pred_antagonism35)

In [152]:
cm2 = confusion_matrix(y_test_antagonism, y_pred_antagonism35, labels = ["Conciliant", "Résistant", "Opposant","Irréconciliant"])
print(cm2)
print("Précision:", precision_score(y_test_antagonism, y_pred_antagonism35, average='macro'))
print("Rappel:", recall_score(y_test_antagonism, y_pred_antagonism35, average='macro'))
print("Score F1:", f1_score(y_test_antagonism, y_pred_antagonism35, average='macro'))
print("Accuracy:", accuracy_score(y_test_antagonism, y_pred_antagonism35))

[[25  1  0  0]
 [ 5 28  0  0]
 [ 3 13  6  2]
 [ 0  1  2 14]]
Précision: 0.7584346370683581
Rappel: 0.720888180447004
Score F1: 0.7019461452166627
Accuracy: 0.73


In [153]:
predictions_roberta_large_zeroshot_v2_0_3 = pd.DataFrame({
    'synergy': y_pred_synergy35,
    'antagonism': y_pred_antagonism35
})

### Tache 3 : classification finale

In [154]:
predictions_roberta_large_zeroshot_v2_0_3['class'] =predictions_roberta_large_zeroshot_v2_0_3.apply(assign_group, axis=1)

In [155]:
y_pred35 = predictions_roberta_large_zeroshot_v2_0_3["class"]

In [156]:
cm3 = confusion_matrix(y_test, y_pred35, labels = ["Passif", "Aligné", "Engagé","Moteur","Déchiré", "Opposant", "Irréductible"])
print(cm3)
print("Précision:", precision_score(y_test, y_pred35, average='macro'))
print("Rappel:", recall_score(y_test, y_pred35, average='macro'))
print("Score F1:", f1_score(y_test, y_pred35, average='macro'))
print("Accuracy:", accuracy_score(y_test, y_pred35))

[[27  1  0  0  0  0  0]
 [ 2  4  1  0  0  0  0]
 [ 0  2  3  0  0  0  0]
 [12  1  2 10  2  1  0]
 [ 0  0  0  1  5  0  0]
 [10  0  0  1  0  4  0]
 [ 0  0  0  0  1  2  8]]
Précision: 0.6513105242096839
Rappel: 0.6171614100185528
Score F1: 0.5974799319521885
Accuracy: 0.61


## Modèle 6 : MoritzLaurer/bge-m3-zeroshot-v2.0

### Tache 1 : classification selon son niveau de synergie

In [157]:
y_pred_synergy36 = []

In [158]:
start = time.time()
for text in avis["Sample_text"]:
    result = classifier6(text, text_synergy_labels3, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_synergy36.append("Minimaliste")
    elif result == "2":
        y_pred_synergy36.append("Intéressé")
    elif result == "3" :
        y_pred_synergy36.append("Coopérant")
    else :
        y_pred_synergy36.append("Engagé")
end = time.time()
execution_time361 = end - start
execution_time361 = execution_time361/60

print(f'Temps d\'exécution : {execution_time361:.2f}minutes')

Temps d'exécution : 4.53minutes


In [159]:
y_pred_synergy36 = np.array(y_pred_synergy36 )

In [160]:
cm = confusion_matrix(y_test_synergy, y_pred_synergy36, labels = ["Minimaliste", "Intéressé", "Coopérant","Engagé"])
print(cm)
print("Précision:", precision_score(y_test_synergy, y_pred_synergy36, average='macro'))
print("Rappel:", recall_score(y_test_synergy, y_pred_synergy36, average='macro'))
print("Score F1:", f1_score(y_test_synergy, y_pred_synergy36, average='macro'))
print("Accuracy:", accuracy_score(y_test_synergy, y_pred_synergy36))

[[15  2  4  9]
 [ 3  0  6 15]
 [ 1  0 10  8]
 [ 0  0  5 22]]
Précision: 0.3992202729044834
Rappel: 0.46028265107212474
Score F1: 0.40250005726196203
Accuracy: 0.47


### Tache 2 : classification selon son niveau d'antagonisme

In [164]:
y_pred_antagonism36 = []

In [165]:
start = time.time()
for text in avis["Sample_text"]:
    result = classifier6(text, text_antagonism_labels3, multi_label = False )
    result = result['labels'][0].split()[0]
    if result == "1":
        y_pred_antagonism36.append("Conciliant")
    elif result == "2":
        y_pred_antagonism36.append("Résistant")
    elif result == "3" :
        y_pred_antagonism36.append("Opposant")
    else :
        y_pred_antagonism36.append("Irréconciliant")
end = time.time()
execution_time362 = end - start
execution_time362 = execution_time362/60

print(f'Temps d\'exécution : {execution_time362:.2f}minutes')

Temps d'exécution : 4.54minutes


In [166]:
y_pred_antagonism36 = np.array(y_pred_antagonism36)

In [167]:
cm2 = confusion_matrix(y_test_antagonism, y_pred_antagonism36, labels = ["Conciliant", "Résistant", "Opposant","Irréconciliant"])
print(cm2)
print("Précision:", precision_score(y_test_antagonism, y_pred_antagonism36, average='macro'))
print("Rappel:", recall_score(y_test_antagonism, y_pred_antagonism36, average='macro'))
print("Score F1:", f1_score(y_test_antagonism, y_pred_antagonism36, average='macro'))
print("Accuracy:", accuracy_score(y_test_antagonism, y_pred_antagonism36))

[[26  0  0  0]
 [20 12  1  0]
 [11  9  4  0]
 [ 4  1  6  6]]
Précision: 0.5838301043219076
Rappel: 0.47081105169340465
Score F1: 0.4460938361987837
Accuracy: 0.48


In [168]:
predictions_bge_m3_zeroshot_v2_0_3 = pd.DataFrame({
    'synergy': y_pred_synergy36,
    'antagonism': y_pred_antagonism36
})

### Tache 3 : classification finale

In [169]:
predictions_bge_m3_zeroshot_v2_0_3['class'] =predictions_bge_m3_zeroshot_v2_0_3.apply(assign_group, axis=1)

In [170]:
y_pred36 = predictions_bge_m3_zeroshot_v2_0_3["class"]

In [171]:
cm3 = confusion_matrix(y_test, y_pred36, labels = ["Passif", "Aligné", "Engagé","Moteur","Déchiré", "Opposant", "Irréductible"])
print(cm3)
print("Précision:", precision_score(y_test, y_pred36, average='macro'))
print("Rappel:", recall_score(y_test, y_pred36, average='macro'))
print("Score F1:", f1_score(y_test, y_pred36, average='macro'))
print("Accuracy:", accuracy_score(y_test, y_pred36))

[[ 7  4 11  5  0  1  0]
 [ 0  4  3  0  0  0  0]
 [ 0  2  3  0  0  0  0]
 [ 1  5 11 11  0  0  0]
 [ 0  1  2  3  0  0  0]
 [ 3  2  3  5  0  2  0]
 [ 0  0  1  1  2  3  4]]
Précision: 0.38859349800526266
Rappel: 0.33017934446505876
Score F1: 0.28167491089323976
Accuracy: 0.31


# Sauvegarde des résultats 

## Modèle 1 : facebook/bart-large-mnli"

In [172]:
#Prompt 1 
predictions_bart_large_mnli_1.to_csv("predictions_bart_large_mnli_1.csv", index = False)
#Prompt 2 
predictions_bart_large_mnli_2.to_csv("predictions_bart_large_mnli_2.csv", index = False)
#Prompt 3 
predictions_bart_large_mnli_3.to_csv("predictions_bart_large_mnli_3.csv", index = False)

## Modèle 2 : 

In [173]:
#Prompt 1 
predictions_deberta_v3_large_zeroshot_v2_0_1.to_csv("predictions_deberta_v3_large_zeroshot_v2_0_1.csv", index = False)
#Prompt 2 
predictions_deberta_v3_large_zeroshot_v2_0_2.to_csv("predictions_deberta_v3_large_zeroshot_v2_0_2.csv", index = False)
#Prompt 3 
predictions_deberta_v3_large_zeroshot_v2_0_3.to_csv("predictions_deberta_v3_large_zeroshot_v2_0_3.csv", index = False)

## Modèle 3:

In [174]:
#Prompt 1 
predictions_deberta_v3_large_zeroshot_v2_0_c_1.to_csv("predictions_deberta_v3_large_zeroshot_v2_0_c_1.csv", index = False)
#Prompt 2 
predictions_deberta_v3_large_zeroshot_v2_0_c_2.to_csv("predictions_deberta_v3_large_zeroshot_v2_0_c_2.csv", index = False)
#Prompt 3 
predictions_deberta_v3_large_zeroshot_v2_0_c_3.to_csv("predictions_deberta_v3_large_zeroshot_v2_0_c_3.csv", index = False)

## Modèle 4 :

In [175]:
#Prompt 1 
predictions_deberta_v3_base_zeroshot_v2_0_1.to_csv("predictions_deberta_v3_base_zeroshot_v2_0_1.csv", index = False)
#Prompt 2 
predictions_deberta_v3_base_zeroshot_v2_0_2.to_csv("predictions_deberta_v3_base_zeroshot_v2_0_2.csv", index = False)
#Prompt 3 
predictions_deberta_v3_base_zeroshot_v2_0_3.to_csv("predictions_deberta_v3_base_zeroshot_v2_0_3.csv", index = False)

## Modèle 5 :

In [176]:
#Prompt 1 
predictions_roberta_large_zeroshot_v2_0_1.to_csv("predictions_roberta_large_zeroshot_v2_0_1.csv", index = False)
#Prompt 2 
predictions_roberta_large_zeroshot_v2_0_2.to_csv("predictions_roberta_large_zeroshot_v2_0_2.csv", index = False)
#Prompt 3 
predictions_roberta_large_zeroshot_v2_0_3.to_csv("predictions_roberta_large_zeroshot_v2_0_3.csv", index = False)

## Modèle 6 :

In [177]:
#Prompt 1 
predictions_bge_m3_zeroshot_v2_0_1.to_csv("predictions_bge_m3_zeroshot_v2_0_1.csv", index = False)
#Prompt 2 
predictions_bge_m3_zeroshot_v2_0_2.to_csv("predictions_bge_m3_zeroshot_v2_0_2.csv", index = False)
#Prompt 3 
predictions_bge_m3_zeroshot_v2_0_3.to_csv("predictions_bge_m3_zeroshot_v2_0_3.csv", index = False)